Alhassan, Abdulaziz and Sevtsuk, Andres, Madina Python Package: Scalable Urban Network Analysis for Modeling Pedestrian and Bicycle Trips in Cities. Available at SSRN: https://ssrn.com/abstract=4748255 or http://dx.doi.org/10.2139/ssrn.4748255

In [147]:
import madina as md
import geopandas as gpd
import pandas as pd
import math
from shapely.geometry import box
import madina.una.tools as una
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

In [ ]:
#collison['DATETIME'] = pd.to_datetime(collison['CRASH DATE'] + ' ' + collison['CRASH TIME'])
#collison21_23 = collison.loc[(collison['DATETIME'] > pd.Timestamp('2021-1-1 00:00'))]
#collison21_23.iloc[:,:-1].to_file('../01_data/03_intermediate/Collision_21_23/Collision.shp', driver='ESRI Shapefile')

In [61]:
nycstreet = gpd.read_file("../01_data/03_intermediate/cut/cut.shp")

In [82]:
collison = gpd.read_file('../01_data/03_intermediate/Collision_21_23/Collision.shp')
collison['DATETIME'] = pd.to_datetime(collison['CRASH DATE'] + ' ' + collison['CRASH TIME'])

----

In [98]:
delta_lat = 2 / 111  
delta_lon = 2 / (111 * math.cos(math.radians(latitude)))

In [171]:
get_previous_index = gpd.GeoDataFrame()

In [174]:
for i in range(0,collison.shape[0]):

    origin = collison.iloc[i:i+1]

    # time
    mask = (collison['DATETIME'] >= origin['DATETIME'].iloc[0] - pd.Timedelta(days=30)) & (collison['DATETIME'] < origin['DATETIME'].iloc[0])
    
    # space
    min_lat = origin['LATITUDE'] - delta_lat
    max_lat = origin['LATITUDE'] + delta_lat
    min_lon = origin['LONGITUDE'] - delta_lon
    max_lon = origin['LONGITUDE'] + delta_lon
    bounding_box = box(min_lon, min_lat, max_lon, max_lat)

    #slice
    destination = collison.loc[mask]
    space = nycstreet[nycstreet['geometry'].intersects(bounding_box)]
    destination = destination[destination['geometry'].within(bounding_box)]

    if destination.shape[0] == 0 or space.shape[0] == 0:

        gdf = origin.copy()
        gdf = gdf.to_crs('EPSG:32619')
        gdf['gravity_to_previous'] = 0
    
    else:

        # weight
        destination['duration'] = (origin['DATETIME'].iloc[0] - destination['DATETIME']).dt.total_seconds() / 3600 # hour
        destination['weight'] = 1/destination['duration'] 

        # prejection
        space = space.to_crs('EPSG:32619')
        origin = origin.to_crs('EPSG:32619')
        destination = destination.to_crs('EPSG:32619')

        # Load Layer
        nyc = md.Zonal()
        nyc.load_layer(name='street', source = space)
        nyc.load_layer('collison', source = origin)
        nyc.load_layer('time_lag', source = destination)

        # Creating a network, and adding origins and destinations
        nyc.create_street_network(source_layer="street")
        nyc.insert_node(label='origin', layer_name="collison")
        nyc.insert_node(label='destination', layer_name="time_lag")

        nyc.create_graph()

        una.accessibility(
            nyc,
            search_radius=3000,
            beta=0.001,
            save_gravity_as='gravity_to_previous',
            destination_weight= 'duration'
        )

        gdf = nyc['collison'].gdf

    get_previous_index = pd.concat([get_previous_index, gdf], ignore_index = True)

In [175]:
get_previous_index

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET,CROSS STRE,OFF STREET,...,CONTRIBU_4,COLLISION_,VEHICLE TY,VEHICLE _1,VEHICLE _2,VEHICLE _3,VEHICLE _4,geometry,DATETIME,gravity_to_previous
0,09/11/2021,9:35,BROOKLYN,11208,40.667202,-73.866500,"(40.667202, -73.8665)",None,None,1211 LORING AVENUE,...,None,4456314,Sedan,None,None,None,None,POINT (88593.066 4513214.943),2021-09-11 09:35:00,16202.516952
1,12/14/2021,8:13,BROOKLYN,11233,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,None,...,None,4486609,None,None,None,None,None,POINT (84399.405 4515242.806),2021-12-14 08:13:00,20042.639869
2,12/14/2021,8:17,BRONX,10475,40.868160,-73.831480,"(40.86816, -73.83148)",None,None,344 BAYCHESTER AVENUE,...,None,4486660,Sedan,Sedan,None,None,None,POINT (92786.584 4535370.284),2021-12-14 08:17:00,4609.842574
3,12/14/2021,21:10,BROOKLYN,11207,40.671720,-73.897100,"(40.67172, -73.8971)",None,None,2047 PITKIN AVENUE,...,None,4487074,Sedan,None,None,None,None,POINT (86033.256 4513860.769),2021-12-14 21:10:00,25428.651450
4,12/14/2021,14:58,MANHATTAN,10017,40.751440,-73.973970,"(40.75144, -73.97397)",3 AVENUE,EAST 43 STREET,None,...,None,4486519,Sedan,Station Wagon/Sport Utility Vehicle,None,None,None,POINT (80035.387 4523080.846),2021-12-14 14:58:00,27034.099215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,03/26/2022,19:00,BROOKLYN,11234,40.617645,-73.920920,"(40.617645, -73.92092)",None,None,1549 EAST 56 STREET,...,None,4513752,Sedan,None,None,None,None,POINT (83682.114 4507967.691),2022-03-26 19:00:00,7650.678422
96,03/26/2022,16:40,BROOKLYN,11217,40.686510,-73.983780,"(40.68651, -73.98378)",None,None,436 ATLANTIC AVENUE,...,None,4513860,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,None,None,None,POINT (78796.544 4515916.399),2022-03-26 16:40:00,16163.861898
97,03/24/2022,18:05,BROOKLYN,11206,40.698310,-73.949800,"(40.69831, -73.9498)",MARCY AVENUE,ELLERY STREET,None,...,None,4514251,Sedan,Station Wagon/Sport Utility Vehicle,None,None,None,POINT (81743.441 4517064.210),2022-03-24 18:05:00,18807.732328
98,03/20/2022,10:34,MANHATTAN,10024,40.778805,-73.974075,"(40.778805, -73.974075)",WEST 76 STREET,CENTRAL PARK WEST,None,...,None,4514302,Sedan,None,None,None,None,POINT (80199.244 4526120.693),2022-03-20 10:34:00,6241.076851
